In [ ]:
import json
import requests
import os
from datetime import datetime
import zipfile
import tempfile
from collections import defaultdict
import time

In [ ]:
JSON_FILE = "./memories_history2020.json"

In [ ]:
# Output folder
output_dir = "snapchat_downloads_renamed"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Load JSON
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
memories = data['Saved Media']

In [ ]:
def extension_from_headers(headers, media_type):
    content_type = headers.get("Content-Type", "").lower()

    if "video/mp4" in content_type:
        return ".mp4"
    if "image/jpeg" in content_type:
        return ".jpg"
    if "image/png" in content_type:
        return ".png"
    if "image/webp" in content_type:
        return ".webp"
    if "application/zip" in content_type:
        return ".zip"

    # fallback if Snapchat does something weird
    if media_type == "Video":
        return ".mp4"
    if media_type == "Image":
        return ".jpg"

    return ""

In [ ]:
dates = [item["Date"] for item in memories]

dates_dict = {}
for date in dates:
    if date not in dates_dict:
        dates_dict[date]=1
    else:
        dates_dict[date]+=1
        
dupes = {}

In [ ]:
for item in memories:
    
    url = item["Media Download Url"]
    media_type = item["Media Type"]
    date_string = item["Date"]

    # clean date for filename
    date = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S UTC")
    
    if dates_dict[date_string] == 1:
        base_name = date.strftime("%Y-%m-%d_%H%M%S")
    else:
        if date in dupes:
            dupes[date]+=1
        else:
            dupes[date]=1
        base_name = date.strftime("%Y-%m-%d_%H%M%S") + "_" + str(dupes[date])

    try:
        response = requests.get(url, allow_redirects=True)
        response.raise_for_status()
    except:
        if response.status_code >= 500:
            time.sleep(5)
            try:
                response = requests.get(url, allow_redirects=True)
                response.raise_for_status()
            except:
                print("Could not save: ", filename)
                continue
            

    ext = extension_from_headers(response.headers, media_type)
    filename = os.path.join(output_dir, base_name + ext)
    
    with open(filename, "wb") as f:
        f.write(response.content)

    print("Saved:", filename)
